<a href="https://colab.research.google.com/github/slsabilaAura/Tugas-Akhir/blob/main/LDA_Topik_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install gensim

In [ ]:
# prompt: import file form drive

from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
import pandas as pd
# Save the preprocessed DataFrames to new CSV files
proposal_df = pd.read_csv('/content/drive/MyDrive/Skripsi3/Dataset/processed_proposalC.csv')
expert_df = pd.read_csv('/content/drive/MyDrive/Skripsi3/Dataset/processed_expertC.csv')

In [ ]:
import numpy as np
import gensim
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
import ast
from gensim.models import Phrases
from gensim.models.phrases import Phraser

# Load Data

In [ ]:
def convert_to_list(text):
    try:
        return ast.literal_eval(text) if isinstance(text, str) else text
    except:
        return []


proposal_df["processed"] = proposal_df["stemmed"].apply(convert_to_list)
expert_df["processed"] = expert_df["stemmed"].apply(convert_to_list)

# Gabungkan semua dokumen untuk membuat satu kamus bersama
documents_all = proposal_df["processed"].tolist() + expert_df["processed"].tolist()
dictionary_all = Dictionary(documents_all)

dictionary_all.filter_extremes(no_below=5, no_above=0.5)

proposal_corpus = [dictionary_all.doc2bow(doc) for doc in proposal_df["processed"]]
expert_corpus = [dictionary_all.doc2bow(doc) for doc in expert_df["processed"]]

corpus_all= proposal_corpus + expert_corpus

In [ ]:
print(len(documents_all))
print(len(corpus_all))
print(len(dictionary_all))

609
609
3086


In [ ]:
# # Misalnya kita ambil satu dokumen dari corpus_all
# doc_bow = corpus_all[0]

# # # Tampilkan representasi kata dari dokumen tersebut
# for word_id, freq in doc_bow:
#     word = dictionary_all[word_id]  # ambil kata dari kamus berdasarkan indeks
#     print(f"('{word}', {freq})")

# Model LDA

In [ ]:
# import time
# import pandas as pd
# import pandas as pd
# from gensim.models.ldamodel import LdaModel
# from gensim.models.coherencemodel import CoherenceModel

# # List untuk menyimpan hasil eksperimen
# results_lda = []


# for num_topics in range(1, 51):
#     start_time = time.time()  # Mulai timer

#     # Train LDA model
#     lda_model_lda = LdaModel(
#         corpus=corpus_all,
#         id2word=dictionary_all,
#         num_topics=num_topics,
#         alpha=0.5,   # Bisa diganti dengan nilai tetap
#         eta=0.01,     # Bisa diganti dengan nilai tetap
#         passes=50,
#         random_state=42,
#         iterations= 400,
#     )

#     # Evaluasi dengan coherence score
#     coherence_model_lda= CoherenceModel(
#         model=lda_model_lda,
#         texts=proposal_df['processed'].tolist() + expert_df['processed'].tolist(),
#         dictionary=dictionary_all,
#         coherence='c_v'
#     )
#     coherence_score_lda = coherence_model_lda.get_coherence()

#     # Evaluasi Topic Diversity
#     top_words_per_topic_lda = [lda_model_lda.show_topic(topic_id, topn=20) for topic_id in range(num_topics)]
#     unique_words_lda = set([word for topic in top_words_per_topic_lda for word, _ in topic])
#     total_words_lda = num_topics * 20  # 10 kata per topik
#     topic_diversity_lda = len(unique_words_lda) / total_words_lda if total_words_lda > 0 else 0

#     # Hitung waktu eksekusi
#     time_taken_lda = time.time() - start_time

#     # Simpan hasil dalam list
#     results_lda.append([num_topics, coherence_score_lda, topic_diversity_lda, time_taken_lda])
#     print(f"Topics: {num_topics} | Coherence: {coherence_score_lda:.4f} | Diversity: {topic_diversity_lda:.4f} | Time: {time_taken_lda:.2f} sec")

# # Simpan ke DataFrame
# results_looping_df = pd.DataFrame(results_lda, columns=['num_topics', 'coherence_score_lda', 'topic_diversity_lda', 'time_taken_lda'])

In [ ]:
# results_looping_df.to_csv('/content/drive/MyDrive/Skripsi4/topik/dictionary/looping_baru,3koma3.csv', index=False, encoding='utf-8-sig')

In [ ]:
from itertools import combinations

num_topics = 30  # atau optimalisasi via coherence

lda_model = LdaModel(
    corpus=corpus_all,
    id2word=dictionary_all,
    num_topics=num_topics,
    passes=50,
    random_state=42,
    iterations= 400,
    alpha=0.5,
    eta=0.01

)

# coherence_model_lda = CoherenceModel(model=lda_model, texts=proposal_df['processed'].tolist() + expert_df['processed'].tolist(), dictionary=dictionary_all, coherence='c_v')
# coherence_score = coherence_model_lda.get_coherence()
# print(f"Coherence Score: {coherence_score}")


In [ ]:
# Simpan dictionary
lda_model.save("lda_model_30.model")
dictionary_all.save("dictionary_lda.dict")

In [ ]:
import pandas as pd

data = []

for topic_id in range(num_topics):
    top_words = lda_model.show_topic(topic_id, topn=25)
    words = [word for word, prob in top_words]
    probs = [round(prob, 4) for word, prob in top_words]

    data.append({
        'Topik': topic_id + 1,
        'Kata Kunci': ', '.join(words),
        'Probabilitas': ', '.join(map(str, probs))
    })

df = pd.DataFrame(data)

# Tampilkan hasil
# print(df)

# Simpan ke CSV jika diinginkan
df.to_csv('/content/drive/MyDrive/Skripsi4/dictionary/topik_kata_dan_probabilitas_30.csv', index=False, encoding='utf-8-sig')



In [ ]:
# import matplotlib.pyplot as plt

# for topic_id in range(num_topics):
#     plt.figure(figsize=(8, 4))
#     terms = lda_model.show_topic(topic_id, topn=4)
#     words = [w for w, _ in terms]
#     scores = [s for _, s in terms]

#     plt.barh(words, scores)
#     plt.gca().invert_yaxis()
#     plt.title(f"Topik {topic_id}")
#     plt.xlabel("Probabilitas")
#     plt.tight_layout()
#     plt.show()


In [ ]:
from gensim.matutils import sparse2full

# Fungsi untuk mendapatkan dense topic vector
def get_topic_vector(lda_model, dictionary, document, num_topics):
    bow = dictionary.doc2bow(document)
    topic_dist = lda_model.get_document_topics(bow, minimum_probability=0.00001)
    return sparse2full(topic_dist, num_topics)

# Ambil jumlah topik dari model LDA
num_topics = lda_model.num_topics

# Hitung topik vektor dalam bentuk dense array (untuk expert)
expert_df["topic_vector"] = [
    get_topic_vector(lda_model, dictionary_all, doc, num_topics)
    for doc in expert_df["processed"]
]

# Hitung topik vektor dalam bentuk dense array (untuk proposal)
proposal_df["topic_vector"] = [
    get_topic_vector(lda_model, dictionary_all, doc, num_topics)
    for doc in proposal_df["processed"]
]


In [ ]:
import numpy as np
import pandas as pd

topic_matrix_expert = np.vstack(expert_df["topic_vector"])
topic_df = pd.DataFrame(
    topic_matrix_expert,
    columns=[f"topic_{i+1}" for i in range(lda_model.num_topics)]
)

topic_df["research_id"] = expert_df["research_id"].values

# Set kolom id_dosen sebagai index
topic_df.set_index("research_id", inplace=True)

# (Opsional) Tampilkan 5 baris pertama untuk verifikasi
# print(topic_df.head())

# print(topic_df)
topic_df.to_csv('/content/drive/MyDrive/Skripsi4/dictionary/vektorExpert_30.csv', index=False)


In [ ]:
topic_matrix_proposal = np.vstack(proposal_df["topic_vector"])
topic_proposal_df = pd.DataFrame(topic_matrix_proposal, columns=[f"topic_{i+1}" for i in range(lda_model.num_topics)])
topic_proposal_df["proposal_id"] = proposal_df["proposal_id"].values

# Set kolom id_dosen sebagai index
topic_proposal_df.set_index("proposal_id", inplace=True)

# print(topic_proposal_df)
topic_proposal_df.to_csv('/content/drive/MyDrive/Skripsi4/dictionary/vektorProposal_30.csv', index=False)

# Kemiripan

In [ ]:
proposal_df["tahun"] = proposal_df["proposal_year"].astype(int)
expert_df["pub_year"] = expert_df["research_pub_year"].astype(int)


def similarity_m_to_d(proposal_vector, expert_vector):
    numerator = np.dot(proposal_vector, expert_vector)
    denominator = (np.linalg.norm(proposal_vector) + 1e-10)
    return numerator / (denominator)


def similarity_d_to_m(expert_vector, proposal_vector):
    numerator = np.dot(expert_vector, proposal_vector)
    denominator = (np.linalg.norm(expert_vector) + 1e-10)
    return numerator / (denominator)

 # denominator = np.linalg.norm(expert_vector)

def time_decay(year_prop, year_ex, t=1, gamma=0.1):
    decay = 1 - ((year_prop - year_ex) / t) * gamma
    return max(decay, 0.0)  # tidak boleh negatif

# mapping Dosen dengan ID Dosen

dosen_id_map = pd.read_csv("/content/drive/MyDrive/Skripsi3/Dataset/mapping.csv")  # pastikan kolom: expert_id, expert_name
dosen_id_map["expert_name"] = dosen_id_map["expert_name"].str.strip().str.lower()


def explode_authors_with_weights(df, dosen_id_map):
    rows = []

    # Normalisasi nama dosen agar cocok
    dosen_id_map = dosen_id_map.copy()
    dosen_id_map["expert_name"] = dosen_id_map["expert_name"].str.strip().str.lower()

    for _, row in df.iterrows():
        authors = row.get("authors", [])
        # Bersihkan nama kosong atau NaN
        authors = [a for a in authors if isinstance(a, str) and a.strip() != ""]

        num_authors = len(authors)
        for idx, name in enumerate(authors):
            name_clean = name.strip().lower()

            if num_authors == 1:
                weight = 1.0
            else:
                weight = 0.6 if idx == 0 else 0.4 / (num_authors - 1)

            new_row = row.to_dict()
            new_row["name"] = name
            new_row["author_position"] = idx + 1
            new_row["num_authors"] = num_authors
            new_row["author_weight"] = round(weight, 4)

            matched = dosen_id_map[dosen_id_map["expert_name"] == name_clean]
            new_row["expert_id"] = matched["expert_id"].values[0] if not matched.empty else None

            rows.append(new_row)

    return pd.DataFrame(rows)


# Gabungkan author_1 sampai author_6 jadi list
author_cols = ["author_1", "author_2", "author_3", "author_4", "author_5", "author_6"]
expert_df["authors"] = expert_df[author_cols].values.tolist()

# Hapus duplikat berdasarkan research_id
expert_df = expert_df.drop_duplicates(subset=["research_id"]).copy()

# Jalankan explode
expert_df = explode_authors_with_weights(expert_df, dosen_id_map)

# Opsional: hanya simpan baris dengan expert_id valid
expert_df = expert_df[expert_df["expert_id"].notna()]




In [ ]:
expert_df.to_csv('/content/drive/MyDrive/Skripsi4/dictionary/exploded.csv', index=False)

In [ ]:
import pandas as pd
import ast

# Load data
df = pd.read_csv("/content/drive/MyDrive/Skripsi4/dictionary/exploded.csv")

import re
import ast

def fix_vector(text):
    if isinstance(text, str):
        # Tambahkan koma antara angka yang dipisah spasi
        text_fixed = re.sub(r'(?<=\d)\s+(?=\d)', ', ', text.strip())
        try:
            return ast.literal_eval(text_fixed)
        except:
            return []
    return text

df["topic_vector"] = df["topic_vector"].apply(fix_vector)

df["topic_vector"].apply(lambda x: len(x)).value_counts()

df.to_csv("/content/drive/MyDrive/Skripsi4/dictionary/exploded_clean.csv", index=False)


## Directed M->D

In [ ]:
def compute_od_m_to_d(proposals_df, experts_df):
    all_results = []

    for _, proposal in proposals_df.iterrows():
        proposal_vector = proposal["topic_vector"]
        mahasiswa = proposal["student_id"]
        id_proposal = proposal["proposal_id"]
        tahun_proposal = proposal["tahun"]

        for _, expert in expert_df.iterrows():
            expert_vector = expert["topic_vector"]
            dosen = expert["name"]
            id_dosen = expert["expert_id"]
            id_penelitian = expert["research_id"]
            tahun_penelitian = expert["pub_year"]
            weight = expert.get("author_weight")
            position= expert["author_position"]

            sim_mahasiswa = similarity_m_to_d(proposal_vector, expert_vector)
            score = sim_mahasiswa * weight

            all_results.append({
                "id_proposal": id_proposal,
                "id_penelitian": id_penelitian,
                "mahasiswa": mahasiswa,
                "dosen": dosen,
                "id_dosen": id_dosen,
                "posisi_author": position,
                "author_weight":weight,
                "tahun_proposal": tahun_proposal,
                "tahun_penelitian": tahun_penelitian,
                "OD(M→D)": round(score, 4)
            })
    df_scores = pd.DataFrame(all_results)
    # df_scores = df_scores.loc[df_scores.groupby(["id_proposal", "id_dosen"])["OD(M→D)"].idxmax()].reset_index(drop=True)
    df_scores = df_scores.sort_values(by=["id_proposal", "OD(M→D)"], ascending=[True, False])
    return df_scores


In [ ]:
od_m2d_df = compute_od_m_to_d(proposal_df, expert_df)
# print(od_m2d_df[["id_proposal", "mahasiswa","id_penelitian","dosen","id_dosen","posisi_author","author_weight", "OD(M→D)"]].head(30))

od_m2d_df.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/od_m2d_df_topik.csv", index=False)

## TOD M->D

In [ ]:
# Hitung TOD(M→D)
def compute_tod_m_to_d(od_df, t=1, gamma=0.1, max_year_diff=5):
    od_df = od_df.copy()

    # Hitung selisih tahun
    od_df["selisih_tahun"] = od_df["tahun_proposal"] - od_df["tahun_penelitian"]

    # Filter: proposal harus lebih baru dari publikasi, dan selisih maksimal 5 tahun
    od_df = od_df[(od_df["selisih_tahun"] >= 0) & (od_df["selisih_tahun"] <= max_year_diff)].copy()

    # Hitung time decay factor
    od_df["time_decay_factor"] = od_df.apply(
        lambda row: time_decay(row["tahun_proposal"], row["tahun_penelitian"], t=t, gamma=gamma),
        axis=1
    )

    # Hitung TOD(M→D)
    od_df["TOD(M→D)"] = (od_df["OD(M→D)"] * od_df["time_decay_factor"]).round(4)

    return od_df



# # Hitung TOD(M→D) sebelum di sortir
# tod_m2d_before_sorted= compute_tod_m_to_d(od_m2d_df, t=1, gamma=0.1)

# # Lihat contoh hasil
# print(tod_m2d_before_sorted[["id_proposal", "id_penelitian","dosen","id_dosen", "OD(M→D)", "tahun_proposal","tahun_penelitian","selisih_tahun", "time_decay_factor", "TOD(M→D)"]].head(5))

# tod_m2d_before_sorted.to_csv("/kaggle/working/tod_m2d_before_sorted.csv", index=False)



In [ ]:
# Hitung TOD(M→D) Disortir
tod_m2d_df= compute_tod_m_to_d(od_m2d_df, t=1, gamma=0.1, max_year_diff=5)

tod_m2d_df = (
    tod_m2d_df
    .loc[tod_m2d_df.groupby(["id_proposal", "id_dosen"])["TOD(M→D)"].idxmax()]
    .reset_index(drop=True)
    .sort_values(by=["id_proposal", "TOD(M→D)"], ascending=[True, False])
)


# Lihat contoh hasil

# print(tod_m2d_df[["id_proposal","mahasiswa", "id_penelitian","dosen","id_dosen", "OD(M→D)", "tahun_proposal","tahun_penelitian","selisih_tahun", "time_decay_factor", "TOD(M→D)"]].head(5))

# tod_m2d_df.to_csv("/kaggle/working/tod_m2d_df.csv", index=False)
tod_m2d_df.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/tod_m2d_df_topik.csv", index=False)

In [ ]:
# dupes_todm_d = tod_m2d_df[tod_m2d_df.duplicated(subset=["id_proposal","dosen", "TOD(M→D)"], keep=False)]
# print(dupes_todm_d)

# OD D->M

In [ ]:
def compute_od_d_to_m(proposals_df, experts_df):
    results = []

    for _, expert in experts_df.iterrows():
        expert_vector = expert["topic_vector"]
        dosen = expert["name"]
        id_dosen = expert["expert_id"]
        id_penelitian = expert["research_id"]
        weight = expert["author_weight"]
        position = expert["author_position"]
        tahun_penelitian = expert["pub_year"]

        for _, proposal in proposals_df.iterrows():
            proposal_vector = proposal["topic_vector"]
            mahasiswa = proposal["student_id"]
            id_proposal = proposal["proposal_id"]
            tahun_proposal = proposal["tahun"]

            # # Tambahkan filter tahun
            selisih_tahun = tahun_proposal - tahun_penelitian
            if 0 < selisih_tahun <= 5 and tahun_penelitian <= tahun_proposal:
               sim_dosen = similarity_d_to_m(expert_vector, proposal_vector)
               score = sim_dosen * weight
               results.append({
                    "id_proposal": id_proposal,
                    "id_penelitian": id_penelitian,
                    "mahasiswa": mahasiswa,
                    "dosen": dosen,
                    "id_dosen": id_dosen,
                    "posisi_author": position,
                    "author_weight": weight,
                    "tahun_proposal": tahun_proposal,
                    "tahun_penelitian": tahun_penelitian,
                    "OD(D→M)": round(score, 4),
                })

    df_scores = pd.DataFrame(results)
    df_scores = df_scores.loc[df_scores.groupby(["id_proposal","id_dosen"])["OD(D→M)"].idxmax()].reset_index(drop=True)
    df_scores = df_scores.sort_values(by=["id_dosen", "OD(D→M)"], ascending=[True, False])

    return df_scores


In [ ]:
#Setelah di Sorted
# od_d2m_df = (
#     od_d2m_before_sorted
#     .sort_values(by=["id_proposal", "OD(D→M)"], ascending=[True, False])
#     # .drop_duplicates(subset=["id_proposal"], keep="first")  #nnti comment ini
#     .reset_index(drop=True)
# )

# Filter nilai di atas 0.3
# od_d2m_df = od_d2m_df.loc[od_d2m_df.groupby(["id_proposal", "id_dosen"])["OD(D→M)"].idxmax()].reset_index(drop=True).sort_values(by=["id_proposal", "OD(D→M)"], ascending=[True, False])

od_d2m_df = compute_od_d_to_m(proposal_df, expert_df)
# print(od_d2m_df[["id_proposal", "mahasiswa","id_penelitian", "dosen", "id_dosen","posisi_author","author_weight","OD(D→M)"]].head(5))

# od_d2m_df.to_csv("/kaggle/working/od_d2m_df.csv", index=False)
od_d2m_df.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/od_d2m_df_topik.csv", index=False)


# Overlap

In [ ]:
# def combine_overlap_scores(df_m2d, df_d2m):
#     # Ambil Top-N dari masing-masing arah
#     top_m2d = df_m2d.groupby("id_proposal")[["id_proposal","mahasiswa", "id_dosen", "dosen", "TOD(M→D)"]]
#     top_d2m = df_d2m.groupby("id_proposal")[["id_proposal","mahasiswa", "id_dosen", "dosen", "OD(D→M)"]]

#     # Outer join untuk semua pasangan top-n
#     merged = pd.merge(top_m2d, top_d2m, on=["id_proposal", "id_dosen"], how="outer")

#     # Tambahkan kolom nama dosen jika hilang (dari M→D arah saja)
#     if "dosen_x" in merged.columns:
#         merged["dosen"] = merged["dosen_x"].combine_first(merged.get("dosen_y"))
#     elif "dosen" not in merged.columns:
#         merged["dosen"] = None

#     if "mahasiswa_x" in merged.columns:
#         merged["mahasiswa"] = merged["mahasiswa_x"].combine_first(merged.get("mahasiswa_y"))
#     elif "mahasiswa" not in merged.columns:
#         merged["mahasiswa"] = None


#     # Ganti NaN skor dengan 0 agar bisa dihitung rata-ratanya
#     merged["TOD(M→D)"] = merged["TOD(M→D)"].fillna(0)
#     merged["OD(D→M)"] = merged["OD(D→M)"].fillna(0)

#     # Tandai overlap jika dosen muncul di kedua arah
#     merged["overlap"] = (merged["TOD(M→D)"] > 0) & (merged["OD(D→M)"] > 0)

#     # Hitung skor rata-rata (hanya jika overlap)
#     merged["skor_rata2"] = merged.apply(
#         lambda row: (row["TOD(M→D)"] + row["OD(D→M)"]) / 2 if row["overlap"] else 0, axis=1
#     )

#     # Ambil skor tertinggi per proposal dan dosen
#     final_scores = merged.sort_values(by=["id_proposal", "skor_rata2"], ascending=[True, False])
#     final_scores = final_scores[["id_proposal", "mahasiswa","dosen", "id_dosen", "TOD(M→D)","OD(D→M)", "skor_rata2", "overlap"]]

#     return final_scores

# df_final = combine_overlap_scores(tod_m2d_df, od_d2m_df)
# # df_final = combine_overlap_scores(tod_m2d_before_sorted, od_d2m_before_sorted, top_n=10)

# # idx = df_final.groupby(["id_proposal", "id_dosen"])["skor_rata2"].idxmax()
# # df_final = df_final.loc[idx]

# # print(df_final.head(5))

# df_final.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/overlap_directed_topik.csv", index=False)

In [ ]:
import pandas as pd

def combine_overlap_scores(df_m2d, df_d2m):
    # Ambil semua kolom yang dibutuhkan dari masing-masing arah
    top_m2d = df_m2d[["id_proposal", "mahasiswa", "id_dosen", "dosen", "TOD(M→D)"]]
    top_d2m = df_d2m[["id_proposal", "mahasiswa", "id_dosen", "dosen", "OD(D→M)"]]

    # Gabungkan kedua dataframe berdasarkan id_proposal dan id_dosen
    merged = pd.merge(top_m2d, top_d2m, on=["id_proposal", "id_dosen"], how="outer", suffixes=('_m2d', '_d2m'))

    # Gabungkan kolom nama mahasiswa dan dosen (dari salah satu sisi)
    merged["mahasiswa"] = merged["mahasiswa_m2d"].combine_first(merged["mahasiswa_d2m"])
    merged["dosen"] = merged["dosen_m2d"].combine_first(merged["dosen_d2m"])

    # Ganti NaN skor dengan 0
    merged["TOD(M→D)"] = merged["TOD(M→D)"].fillna(0)
    merged["OD(D→M)"] = merged["OD(D→M)"].fillna(0)

    # Tandai overlap jika skor dari dua arah ada (lebih dari 0)
    merged["overlap"] = (merged["TOD(M→D)"] > 0) & (merged["OD(D→M)"] > 0)

    # Hitung skor rata-rata hanya jika overlap, jika tidak maka 0
    merged["skor_rata2"] = merged.apply(
        lambda row: (row["TOD(M→D)"] + row["OD(D→M)"]) / 2 if row["overlap"] else 0, axis=1
    )

    # Urutkan berdasarkan skor rata-rata tertinggi untuk setiap proposal
    final_scores = merged.sort_values(by=["id_proposal", "skor_rata2"], ascending=[True, False])

    # Pilih kolom akhir yang relevan
    final_scores = final_scores[[
        "id_proposal", "mahasiswa", "dosen", "id_dosen",
        "TOD(M→D)", "OD(D→M)", "skor_rata2", "overlap"
    ]]

    return final_scores

# Contoh penggunaan:
df_final = combine_overlap_scores(tod_m2d_df, od_d2m_df)
df_final.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/overlap_directed_topik.csv", index=False)


# Ranking

In [ ]:
def combine_overlap_scores_with_ranking(df_m2d, df_d2m):
    # Ambil semua skor dari kedua arah
    m2d = df_m2d[["id_proposal", "id_dosen","mahasiswa", "dosen", "TOD(M→D)"]]
    d2m = df_d2m[["id_proposal","id_dosen","mahasiswa","dosen", "OD(D→M)"]]

    # Outer join agar semua kombinasi muncul
    merged = pd.merge(m2d, d2m, on=["id_proposal", "id_dosen"], how="outer")

    # Tambahkan kolom nama dosen jika hilang (dari M→D arah saja)
    if "dosen_x" in merged.columns:
        merged["dosen"] = merged["dosen_x"].combine_first(merged.get("dosen_y"))
    elif "dosen" not in merged.columns:
        merged["dosen"] = None

    if "mahasiswa_x" in merged.columns:
        merged["mahasiswa"] = merged["mahasiswa_x"].combine_first(merged.get("mahasiswa_y"))
    elif "mahasiswa" not in merged.columns:
        merged["mahasiswa"] = None

    # Isi nilai NaN dengan 0 untuk penggabungan skor
    merged["TOD(M→D)"] = merged["TOD(M→D)"].fillna(0)
    merged["OD(D→M)"] = merged["OD(D→M)"].fillna(0)

    # Overlap = muncul di kedua arah
    merged["overlap"] = (merged["TOD(M→D)"] > 0) & (merged["OD(D→M)"] > 0)

    # Skor rata-rata jika overlap
    merged["skor_rata2"] = merged.apply(
        lambda row: (row["TOD(M→D)"] + row["OD(D→M)"]) / 2 if row["overlap"] else 0, axis=1
    )

       # Hitung ranking per proposal berdasarkan skor rata-rata (tanpa groupby + agg)
    merged["rank"] = merged.groupby("id_proposal")["skor_rata2"]\
                           .rank(ascending=False, method="dense")\
                           .astype(int)

    # Ambil kolom yang diinginkan dan urutkan
    result = merged.sort_values(by=["id_proposal", "rank"])[
        ["id_proposal","mahasiswa", "dosen", "id_dosen", "TOD(M→D)", "OD(D→M)", "skor_rata2", "overlap", "rank"]
    ]


    return result.sort_values(by=["id_proposal", "rank"])[
        ["id_proposal","mahasiswa", "dosen", "id_dosen", "TOD(M→D)", "OD(D→M)", "skor_rata2", "overlap", "rank"]
    ]


df_peringkat = combine_overlap_scores_with_ranking(tod_m2d_df, od_d2m_df)
# Filter hanya yang overlap == True
df_overlap_true = df_peringkat[df_peringkat["overlap"] == True]
# print(df_overlap_true.head(20))

df_overlap_true.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/rank_overlap_true_kata.csv", index=False)


In [ ]:
from collections import defaultdict

from collections import defaultdict

# 1. Inisialisasi count untuk rank 1
rank1_count_directed = defaultdict(int)
final_assignment_directed = []

# 2. Tetapkan rank 1 dengan batas 15 kali per dosen
for pid in df_overlap_true["id_proposal"].unique():
    candidates = df_overlap_true[df_overlap_true["id_proposal"] == pid]
    candidates = candidates.sort_values(by="skor_rata2", ascending=False)

    assigned_rank1 = False
    for _, row in candidates.iterrows():
        dosen_id = row["id_dosen"]
        if rank1_count_directed[dosen_id] < 15:
            rank1_count_directed[dosen_id] += 1
            row_data = row.to_dict()
            row_data["rank"] = 1
            row_data["beban"] = rank1_count_directed[dosen_id]
            final_assignment_directed.append(row_data)
            assigned_rank1 = True
            break

    if not assigned_rank1:
        row = candidates.iloc[0].to_dict()
        dosen_id = row["id_dosen"]
        rank1_count_directed[dosen_id] += 1
        row["rank"] = 1
        row["beban"] = rank1_count_directed[dosen_id]
        final_assignment_directed.append(row)

# 3. Buat dataframe dari rank 1
rank1_directed = pd.DataFrame(final_assignment_directed)

# 4. Tambahkan rank 2–17 berdasarkan similarity, excl. dosen yang sudah dipakai di rank 1 untuk proposal yang sama
other_ranks = []

for pid in df_overlap_true["id_proposal"].unique():
    # Dapatkan dosen yang sudah dipakai sebagai rank 1
    used_dosen = rank1_directed[rank1_directed["id_proposal"] == pid]["id_dosen"].tolist()

    # Ambil kandidat lain untuk proposal ini
    candidates = df_overlap_true[(df_overlap_true["id_proposal"] == pid) & (~df_overlap_true["id_dosen"].isin(used_dosen))]
    candidates = candidates.sort_values(by="skor_rata2", ascending=False).reset_index(drop=True)

    for idx, (_, row) in enumerate(candidates.iterrows(), start=2):
        if idx > 17:
            break
        row_data = row.to_dict()
        row_data["rank"] = idx
        row_data["beban"] = rank1_count_directed[row_data["id_dosen"]]  # Beban hanya dihitung dari rank 1
        other_ranks.append(row_data)

# 5. Gabungkan rank1 dan other ranks
df_ranked_filtered = pd.concat([rank1_directed, pd.DataFrame(other_ranks)], ignore_index=True)
df_ranked_filtered = df_ranked_filtered.sort_values(by=["id_proposal", "rank"])

# 🔟 Filter hanya Top 10 dosen per proposal
df_ranked_filtered = df_ranked_filtered[df_ranked_filtered["rank"] <= 17]

# Print the DataFrame
# print(df_ranked_filtered)

# df_ranked_filtered.to_csv("/kaggle/working/df_ranked_filtered_coba.csv", index=False)
df_ranked_filtered.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/beban_directed_topik.csv", index=False)

# Evaluasi

In [ ]:
from sklearn.metrics import precision_score, recall_score

# Load the true labels DataFrame
true_label_df = pd.read_csv("/content/drive/MyDrive/Skripsi3/Dataset/true_labels.csv")

# Gabungkan label benar menjadi list
# true_label_df["true_dosens"] = true_label_df.apply(lambda row: [
#     row["author1"], row["author2"], row["author3"]
# ], axis=1)


# Ubah kolom author, author2, author3 menjadi lowercase
true_label_df["author1"] = true_label_df["examiner_1"].astype(str).str.strip()
true_label_df["author2"] = true_label_df["examiner_2"].astype(str).str.strip()
true_label_df["author3"] = true_label_df["examiner_3"].astype(str).str.strip()



coba

## Evaluasi Baru

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Contoh struktur dummy data untuk demonstrasi perbaikan fungsi (tidak dijalankan secara nyata di sini)
# similarity_cosine_df = pd.read_csv(...)  # Format: id_proposal, rank, similarity_score_akhir, dosen
# true_label_df = pd.read_csv(...)  # Format: proposal_id, author1, author2, author3

def evaluate_ordered_recommendation_directed(rank_all_df, true_label_df, top_ns=[3, 5, 7, 10]):
    summary = []

    for TOP_N in top_ns:
        # Filter Top-N dan urutkan
        top_n_df = rank_all_df[rank_all_df["rank"] <= TOP_N]
        rec_df = top_n_df.sort_values(by=["id_proposal", "rank", "skor_rata2"], ascending=[True, True, False])
        rec_df = rec_df.drop_duplicates(subset=["id_proposal", "rank","skor_rata2"])
        rec_pivot = rec_df.pivot(index="id_proposal", columns="rank", values="dosen").reset_index()
        rec_pivot.columns.name = None
        rec_pivot.columns = ["id_proposal"] + [f"rec_{i}" for i in range(1, len(rec_pivot.columns))]

        # Gabungkan dengan ground truth
        merged_df = pd.merge(
            rec_pivot,
            true_label_df.rename(columns={"proposal_id": "id_proposal"}),
            on="id_proposal",
            how="left"
        )

        # Recall keberadaan (tidak memperhatikan urutan)
        def recall_of_existence(row):
            true_set = {row.get("author1"), row.get("author2"), row.get("author3")}
            pred_set = {row.get(f"rec_{i}") for i in range(1, TOP_N + 1) if row.get(f"rec_{i}")}
            return len(true_set.intersection(pred_set)) / 3

        merged_df[f'recall_of_existence@{TOP_N}'] = merged_df.apply(recall_of_existence, axis=1)

        # Recall per posisi dengan urutan diperhatikan (rec_i harus sama dengan author_i)
        recall_pos = {1: [], 2: [], 3: []}
        for _, row in merged_df.iterrows():
            for pos in [1, 2, 3]:
                examiner = row.get(f'author{pos}')
                rec = row.get(f'rec_{pos}') if pos <= TOP_N else None
                hit = int(pd.notna(examiner) and pd.notna(rec) and examiner == rec)
                recall_pos[pos].append(hit)

        # Tambahkan recall ke DataFrame
        for pos in [1, 2, 3]:
            merged_df[f'recall_pos{pos}_ordered@{TOP_N}'] = recall_pos[pos]

        recall_pos_mean = {pos: np.mean(recall_pos[pos]) for pos in [1, 2, 3]}

        # Euclidean distance antar posisi (penalti posisi meleset)
        distances = []
        for _, row in merged_df.iterrows():
            true_authors = [row.get(f'author{i}') for i in [1, 2, 3]]
            pred_authors = [row.get(f'rec_{i}', None) for i in range(1, TOP_N + 1)]
            distance = 0
            max_penalty = TOP_N
            for i, true_author in enumerate(true_authors):
                if pd.isna(true_author) or true_author == '':
                    continue
                try:
                    pred_pos = pred_authors.index(true_author)
                    pos_diff = pred_pos - i
                    distance += pos_diff ** 2
                except ValueError:
                    distance += max_penalty ** 2
            distances.append(np.sqrt(distance))

        scaler = MinMaxScaler()
        norm_dists = scaler.fit_transform(np.array(distances).reshape(-1, 1)).flatten()
        merged_df[f'norm_euclidean@{TOP_N}'] = norm_dists

        # Ringkasan metrik
        summary.append({
            'Top-N': TOP_N,
            'Mean_Recall_Existence': merged_df[f'recall_of_existence@{TOP_N}'].mean(),
            'Recall_Pos_1_Ordered': recall_pos_mean[1],
            'Recall_Pos_2_Ordered': recall_pos_mean[2],
            'Recall_Pos_3_Ordered': recall_pos_mean[3],
            'Avg_Normalized_Euclidean': np.mean(norm_dists)
        })

    return pd.DataFrame(summary)

result_df_directed = evaluate_ordered_recommendation_directed(df_overlap_true, true_label_df)
# result_df_directed = evaluate_ordered_recommendation_directed(df_ranked_filtered, true_label_df)
print(result_df_directed)



   Top-N  Mean_Recall_Existence  Recall_Pos_1_Ordered  Recall_Pos_2_Ordered  \
0      3               0.340376              0.380282              0.056338   
1      5               0.511737              0.380282              0.056338   
2      7               0.636150              0.380282              0.056338   
3     10               0.781690              0.380282              0.056338   

   Recall_Pos_3_Ordered  Avg_Normalized_Euclidean  
0              0.091549                  0.822388  
1              0.091549                  0.710031  
2              0.091549                  0.625719  
3              0.091549                  0.555407  


## Evaluasi per proposal Baru

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def evaluate_per_proposal_directed(rank_all_df, true_label_df, top_n=3):
    # Filter dan urutkan
    top_n_df = rank_all_df[rank_all_df["rank"] <= top_n]
    rec_df = top_n_df.sort_values(by=["id_proposal", "rank", "skor_rata2"], ascending=[True, True, False])
    rec_df = rec_df.drop_duplicates(subset=["id_proposal", "rank","skor_rata2"])
    rec_pivot = rec_df.pivot(index="id_proposal", columns="rank", values="dosen").reset_index()
    rec_pivot.columns.name = None
    rec_pivot.columns = ["id_proposal"] + [f"rec_{i}" for i in range(1, len(rec_pivot.columns))]

    # Gabung dengan label kebenaran
    merged_df = pd.merge(
        rec_pivot,
        true_label_df.rename(columns={"proposal_id": "id_proposal"}),
        on="id_proposal",
        how="left"
    )

    # Recall of existence (abaikan urutan)
    def recall_of_existence(row):
        true_set = {row.get("author1"), row.get("author2"), row.get("author3")}
        pred_set = {row.get(f"rec_{i}") for i in range(1, top_n + 1) if row.get(f"rec_{i}")}
        return len(true_set.intersection(pred_set)) / 3

    merged_df[f'recall_of_existence@{top_n}'] = merged_df.apply(recall_of_existence, axis=1)

    # Recall berdasarkan posisi (urutan harus sama)
    for pos in [1, 2, 3]:
        merged_df[f'recall_pos{pos}_ordered@{top_n}'] = merged_df.apply(
            lambda row: int(
                pd.notna(row.get(f'author{pos}')) and
                pd.notna(row.get(f'rec_{pos}')) and
                row.get(f'author{pos}') == row.get(f'rec_{pos}')
            ) if pos <= top_n else 0,
            axis=1
        )

    # Euclidean distance penalti posisi
    distances = []
    for _, row in merged_df.iterrows():
        true_authors = [row.get(f'author{i}') for i in [1, 2, 3]]
        pred_authors = [row.get(f'rec_{i}', None) for i in range(1, top_n + 1)]
        distance = 0
        max_penalty = top_n
        for i, true_author in enumerate(true_authors):
            if pd.isna(true_author) or true_author == '':
                continue
            try:
                pred_pos = pred_authors.index(true_author)
                pos_diff = pred_pos - i
                distance += pos_diff ** 2
            except ValueError:
                distance += max_penalty ** 2
        distances.append(np.sqrt(distance))

    # Normalisasi jarak
    scaler = MinMaxScaler()
    norm_dists = scaler.fit_transform(np.array(distances).reshape(-1, 1)).flatten()
    merged_df[f'norm_euclidean@{top_n}'] = norm_dists

    # Ambil kolom evaluasi
    result_df = merged_df[["id_proposal",
                           f'recall_of_existence@{top_n}',
                           f'recall_pos1_ordered@{top_n}',
                           f'recall_pos2_ordered@{top_n}',
                           f'recall_pos3_ordered@{top_n}',
                           f'norm_euclidean@{top_n}']].copy()

    return result_df


eval_per_proposal_directed_3 = evaluate_per_proposal_directed(df_overlap_true, true_label_df, top_n=3)
eval_per_proposal_directed_5 = evaluate_per_proposal_directed(df_overlap_true, true_label_df, top_n=5)
eval_per_proposal_directed_7 = evaluate_per_proposal_directed(df_overlap_true, true_label_df, top_n=7)
eval_per_proposal_directed_10 = evaluate_per_proposal_directed(df_overlap_true, true_label_df, top_n=10)

eval_per_proposal_directed_3.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/hasil_eval_3_directed.csv", index=False)
eval_per_proposal_directed_5.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/hasil_eval_5_directed.csv", index=False)
eval_per_proposal_directed_7.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/hasil_eval_7_directed.csv", index=False)
eval_per_proposal_directed_10.to_csv("/content/drive/MyDrive/Skripsi4/topik/directed/30/hasil_eval_10_directed.csv", index=False)

In [ ]:
dupes = df_overlap_true[df_overlap_true.duplicated(subset=["id_proposal", "rank"], keep=False)]
print(dupes)


     id_proposal mahasiswa                   dosen id_dosen  TOD(M→D)  \
54          P101      S101         Haryono Setiadi      D14    0.0237   
57          P101      S101  Brilyan Hendrasuryawan      D17    0.0237   
111         P104      S104      Dewi Wisnu Wardani       D4    0.0876   
115         P104      S104            Esti Suryani       D8    0.0876   
104         P104      S104       Hasan Dwi Cahyono      D13    0.0510   
105         P104      S104         Haryono Setiadi      D14    0.0510   
344         P118      S118         Bambang Harjito       D1    0.0842   
346         P118      S118                 Winarno      D11    0.0842   
1145         P35       S35         Bambang Harjito       D1    0.0844   
1147         P35       S35                 Winarno      D11    0.0844   
1177         P37       S37         Bambang Harjito       D1    0.0841   
1179         P37       S37                 Winarno      D11    0.0841   
1275         P42       S42                 Winarno 

# Cosine

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Konversi kolom topic_vector menjadi 2D array
expert_vectors = np.vstack(expert_df["topic_vector"].values)
proposal_vectors = np.vstack(proposal_df["topic_vector"].values)

# Hitung similarity
similarity_matrix = cosine_similarity(expert_vectors, proposal_vectors)

print(similarity_matrix)


[[0.02116158 0.19097795 0.1399585  ... 0.02030642 0.02484113 0.05370035]
 [0.02116158 0.19097795 0.1399585  ... 0.02030642 0.02484113 0.05370035]
 [0.01516028 0.02125837 0.02516963 ... 0.06561315 0.05264645 0.05729351]
 ...
 [0.14608201 0.02545677 0.09075104 ... 0.03309006 0.03286479 0.05853668]
 [0.20352237 0.02324828 0.07729209 ... 0.03353393 0.0428044  0.05218332]
 [0.11509452 0.01096047 0.0180661  ... 0.02063077 0.02465363 0.02465597]]


In [ ]:
similarity_cosine_df = []

for i, (_, mahasiswa) in enumerate(proposal_df.iterrows()):
    for j, (_, dosen) in enumerate(expert_df.iterrows()):
        weight = dosen.get("author_weight")
        similarity_cosine_df.append({
            "id_proposal": mahasiswa["proposal_id"],
            "mahasiswa": mahasiswa["student_id"],
            "id_dosen": dosen["expert_id"],
            "id_penelitian": dosen["research_id"],
            "tahun_proposal": mahasiswa["tahun"],
            "tahun_penelitian": dosen["pub_year"],
            "selisih_tahun": mahasiswa["tahun"] - dosen["pub_year"],
            "dosen": dosen["name"],
           "author_position": dosen["author_position"],
            "weight": weight,
            "similarity_score" : similarity_matrix[j, i]


        })

similarity_cosine_df = pd.DataFrame(similarity_cosine_df)

# Filter sesuai kondisi
similarity_cosine_df = similarity_cosine_df[
    (similarity_cosine_df["tahun_proposal"] > similarity_cosine_df["tahun_penelitian"]) &
    (similarity_cosine_df["selisih_tahun"] <= 5)
].copy()


# Tampilkan hasil
# print(similarity_cosine_df[["id_proposal","mahasiswa","id_penelitian", "id_dosen","selisih_tahun", "dosen","weight", "similarity_score" ,"similarity_score_akhir",]])
# similarity_cosine_df.to_csv("/content/drive/MyDrive/Skripsi4/topik/cosine/14/hasil_cosine_topik.csv", index=False)


# Pemeringkatan

In [ ]:
# Ambil baris dengan similarity tertinggi untuk kombinasi unik id_proposal dan id_dosen
idx = similarity_cosine_df.groupby(["id_proposal", "id_dosen"])["similarity_score"].idxmax()
similarity_cosine = similarity_cosine_df.loc[idx]

# Ranking ulang berdasarkan proposal
similarity_cosine_df["rank"] = similarity_cosine_df.groupby("id_proposal")["similarity_score"] \
                                     .rank(method="first", ascending=False).astype(int)

# Urutkan
similarity_cosine_df = similarity_cosine_df.sort_values(["id_proposal", "rank"])
similarity_cosine_df = similarity_cosine_df[similarity_cosine_df["rank"] <= 17]


# Tampilkan
print(similarity_cosine_df[["id_proposal", "mahasiswa","tahun_proposal","id_penelitian", "tahun_penelitian", "id_dosen", "dosen","selisih_tahun", "weight","similarity_score","rank"]])
# similarity_cosine_df.to_csv("/content/drive/MyDrive/Skripsi4/topik/14/cosine/similarity_cosine_kata_baru_setelah filter.csv", index=False)


      id_proposal mahasiswa  tahun_proposal id_penelitian  tahun_penelitian  \
137            P1        S1            2021           R83              2016   
138            P1        S1            2021           R83              2016   
163            P1        S1            2021           R97              2018   
164            P1        S1            2021           R97              2018   
84             P1        S1            2021           R48              2018   
...           ...       ...             ...           ...               ...   
74550         P99       S99            2024           R42              2023   
74815         P99       S99            2024          R202              2023   
74816         P99       S99            2024          R202              2023   
74685         P99       S99            2024          R116              2021   
74686         P99       S99            2024          R116              2021   

      id_dosen              dosen  selisih_tahun  w

In [ ]:
from collections import defaultdict

# 1. Inisialisasi count untuk rank 1
rank1_count = defaultdict(int)
final_assignments = []

# 2. Tetapkan rank 1 dengan batas 15 kali per dosen
for pid in similarity_cosine_df["id_proposal"].unique():
    candidates = similarity_cosine_df[similarity_cosine_df["id_proposal"] == pid]
    candidates = candidates.sort_values(by="similarity_score", ascending=False)

    assigned_rank1 = False
    for _, row in candidates.iterrows():
        dosen_id = row["id_dosen"]
        if rank1_count[dosen_id] < 15:
            rank1_count[dosen_id] += 1
            row_data = row.to_dict()
            row_data["rank"] = 1
            row_data["beban"] = rank1_count[dosen_id]
            final_assignments.append(row_data)
            assigned_rank1 = True
            break

    if not assigned_rank1:
        row = candidates.iloc[0].to_dict()
        dosen_id = row["id_dosen"]
        rank1_count[dosen_id] += 1
        row["rank"] = 1
        row["beban"] = rank1_count[dosen_id]
        final_assignments.append(row)

# 3. Buat dataframe dari rank 1
rank1_df = pd.DataFrame(final_assignments)

# 4. Tambahkan rank 2–17 berdasarkan similarity, excl. dosen yang sudah dipakai di rank 1 untuk proposal yang sama
other_ranks = []

for pid in similarity_cosine_df["id_proposal"].unique():
    # Dapatkan dosen yang sudah dipakai sebagai rank 1
    used_dosen = rank1_df[rank1_df["id_proposal"] == pid]["id_dosen"].tolist()

    # Ambil kandidat lain untuk proposal ini
    candidates = similarity_cosine_df[(similarity_cosine_df["id_proposal"] == pid) & (~similarity_cosine_df["id_dosen"].isin(used_dosen))]
    candidates = candidates.sort_values(by="similarity_score", ascending=False).reset_index(drop=True)

    for idx, (_, row) in enumerate(candidates.iterrows(), start=2):
        if idx > 17:
            break
        row_data = row.to_dict()
        row_data["rank"] = idx
        row_data["beban"] = rank1_count[row_data["id_dosen"]]  # Beban hanya dihitung dari rank 1
        other_ranks.append(row_data)

# 5. Gabungkan rank1 dan other ranks
rank_all_df = pd.concat([rank1_df, pd.DataFrame(other_ranks)], ignore_index=True)
rank_all_df = rank_all_df.sort_values(by=["id_proposal", "rank"])

# 6. Tampilkan hasil
print(rank_all_df[["id_proposal", "mahasiswa", "id_dosen", "dosen", "similarity_score", "rank", "beban"]])
rank_all_df.to_csv("/content/drive/MyDrive/Skripsi4/topik/cosine/30/beban_cosine_topik.csv", index=False)

     id_proposal mahasiswa id_dosen              dosen  similarity_score  \
0             P1        S1       D8       Esti Suryani          0.765764   
142           P1        S1       D5         Abdul Aziz          0.765764   
143           P1        S1       D2            Wiranto          0.754174   
144           P1        S1       D1    Bambang Harjito          0.657577   
145           P1        S1       D3        Umi Salamah          0.579853   
...          ...       ...      ...                ...               ...   
2009         P99       S99      D11            Winarno          0.563516   
2010         P99       S99      D11            Winarno          0.558069   
2011         P99       S99      D13  Hasan Dwi Cahyono          0.558069   
2012         P99       S99      D12            Wiharto          0.416476   
2013         P99       S99       D8       Esti Suryani          0.416476   

      rank  beban  
0        1      1  
142      2      5  
143      3     13  
144    

# Evaluasi Cosine

coba

## Evaluasi Baru

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Contoh struktur dummy data untuk demonstrasi perbaikan fungsi (tidak dijalankan secara nyata di sini)
# similarity_cosine_df = pd.read_csv(...)  # Format: id_proposal, rank, similarity_score_akhir, dosen
# true_label_df = pd.read_csv(...)  # Format: proposal_id, author1, author2, author3

def evaluate_ordered_recommendation_cosine(rank_all_df, true_label_df, top_ns=[3, 5, 7, 10]):
    summary = []

    for TOP_N in top_ns:
        # Filter Top-N dan urutkan
        top_n_df = rank_all_df[rank_all_df["rank"] <= TOP_N]
        rec_df = top_n_df.sort_values(by=["id_proposal", "rank", "similarity_score"], ascending=[True, True, False])
        rec_pivot = rec_df.pivot(index="id_proposal", columns="rank", values="dosen").reset_index()
        rec_pivot.columns.name = None
        rec_pivot.columns = ["id_proposal"] + [f"rec_{i}" for i in range(1, len(rec_pivot.columns))]

        # Gabungkan dengan ground truth
        merged_df = pd.merge(
            rec_pivot,
            true_label_df.rename(columns={"proposal_id": "id_proposal"}),
            on="id_proposal",
            how="left"
        )

        # Recall keberadaan (tidak memperhatikan urutan)
        def recall_of_existence(row):
            true_set = {row.get("author1"), row.get("author2"), row.get("author3")}
            pred_set = {row.get(f"rec_{i}") for i in range(1, TOP_N + 1) if row.get(f"rec_{i}")}
            return len(true_set.intersection(pred_set)) / 3

        merged_df[f'recall_of_existence@{TOP_N}'] = merged_df.apply(recall_of_existence, axis=1)

        # Recall per posisi dengan urutan diperhatikan (rec_i harus sama dengan author_i)
        recall_pos = {1: [], 2: [], 3: []}
        for _, row in merged_df.iterrows():
            for pos in [1, 2, 3]:
                examiner = row.get(f'author{pos}')
                rec = row.get(f'rec_{pos}') if pos <= TOP_N else None
                hit = int(pd.notna(examiner) and pd.notna(rec) and examiner == rec)
                recall_pos[pos].append(hit)

        # Tambahkan recall ke DataFrame
        for pos in [1, 2, 3]:
            merged_df[f'recall_pos{pos}_ordered@{TOP_N}'] = recall_pos[pos]

        recall_pos_mean = {pos: np.mean(recall_pos[pos]) for pos in [1, 2, 3]}

        # Euclidean distance antar posisi (penalti posisi meleset)
        distances = []
        for _, row in merged_df.iterrows():
            true_authors = [row.get(f'author{i}') for i in [1, 2, 3]]
            pred_authors = [row.get(f'rec_{i}', None) for i in range(1, TOP_N + 1)]
            distance = 0
            max_penalty = TOP_N
            for i, true_author in enumerate(true_authors):
                if pd.isna(true_author) or true_author == '':
                    continue
                try:
                    pred_pos = pred_authors.index(true_author)
                    pos_diff = pred_pos - i
                    distance += pos_diff ** 2
                except ValueError:
                    distance += max_penalty ** 2
            distances.append(np.sqrt(distance))

        scaler = MinMaxScaler()
        norm_dists = scaler.fit_transform(np.array(distances).reshape(-1, 1)).flatten()
        merged_df[f'norm_euclidean@{TOP_N}'] = norm_dists

        # Ringkasan metrik
        summary.append({
            'Top-N': TOP_N,
            'Mean_Recall_Existence': merged_df[f'recall_of_existence@{TOP_N}'].mean(),
            'Recall_Pos_1_Ordered': recall_pos_mean[1],
            'Recall_Pos_2_Ordered': recall_pos_mean[2],
            'Recall_Pos_3_Ordered': recall_pos_mean[3],
            'Avg_Normalized_Euclidean': np.mean(norm_dists)
        })

    return pd.DataFrame(summary)

result_df = evaluate_ordered_recommendation_cosine(similarity_cosine_df, true_label_df)
print(result_df)



   Top-N  Mean_Recall_Existence  Recall_Pos_1_Ordered  Recall_Pos_2_Ordered  \
0      3               0.244131              0.309859              0.021127   
1      5               0.380282              0.309859              0.021127   
2      7               0.488263              0.309859              0.021127   
3     10               0.551643              0.309859              0.021127   

   Recall_Pos_3_Ordered  Avg_Normalized_Euclidean  
0              0.070423                  0.710102  
1              0.070423                  0.686285  
2              0.070423                  0.647736  
3              0.070423                  0.616253  


## per proposal baru

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def evaluate_per_proposal_cosine(rank_all_df, true_label_df, top_n=3):
    # Filter dan urutkan
    top_n_df = rank_all_df[rank_all_df["rank"] <= top_n]
    rec_df = top_n_df.sort_values(by=["id_proposal", "rank", "similarity_score"], ascending=[True, True, False])
    rec_pivot = rec_df.pivot(index="id_proposal", columns="rank", values="dosen").reset_index()
    rec_pivot.columns.name = None
    rec_pivot.columns = ["id_proposal"] + [f"rec_{i}" for i in range(1, len(rec_pivot.columns))]

    # Gabung dengan label kebenaran
    merged_df = pd.merge(
        rec_pivot,
        true_label_df.rename(columns={"proposal_id": "id_proposal"}),
        on="id_proposal",
        how="left"
    )

    # Recall of existence (abaikan urutan)
    def recall_of_existence(row):
        true_set = {row.get("author1"), row.get("author2"), row.get("author3")}
        pred_set = {row.get(f"rec_{i}") for i in range(1, top_n + 1) if row.get(f"rec_{i}")}
        return len(true_set.intersection(pred_set)) / 3

    merged_df[f'recall_of_existence@{top_n}'] = merged_df.apply(recall_of_existence, axis=1)

    # Recall berdasarkan posisi (urutan harus sama)
    for pos in [1, 2, 3]:
        merged_df[f'recall_pos{pos}_ordered@{top_n}'] = merged_df.apply(
            lambda row: int(
                pd.notna(row.get(f'author{pos}')) and
                pd.notna(row.get(f'rec_{pos}')) and
                row.get(f'author{pos}') == row.get(f'rec_{pos}')
            ) if pos <= top_n else 0,
            axis=1
        )

    # Euclidean distance penalti posisi
    distances = []
    for _, row in merged_df.iterrows():
        true_authors = [row.get(f'author{i}') for i in [1, 2, 3]]
        pred_authors = [row.get(f'rec_{i}', None) for i in range(1, top_n + 1)]
        distance = 0
        max_penalty = top_n
        for i, true_author in enumerate(true_authors):
            if pd.isna(true_author) or true_author == '':
                continue
            try:
                pred_pos = pred_authors.index(true_author)
                pos_diff = pred_pos - i
                distance += pos_diff ** 2
            except ValueError:
                distance += max_penalty ** 2
        distances.append(np.sqrt(distance))

    # Normalisasi jarak
    scaler = MinMaxScaler()
    norm_dists = scaler.fit_transform(np.array(distances).reshape(-1, 1)).flatten()
    merged_df[f'norm_euclidean@{top_n}'] = norm_dists

    # Ambil kolom evaluasi
    result_df = merged_df[["id_proposal",
                           f'recall_of_existence@{top_n}',
                           f'recall_pos1_ordered@{top_n}',
                           f'recall_pos2_ordered@{top_n}',
                           f'recall_pos3_ordered@{top_n}',
                           f'norm_euclidean@{top_n}']].copy()

    return result_df


eval_per_proposal_cosine_3 = evaluate_per_proposal_cosine(similarity_cosine_df, true_label_df, top_n=3)
eval_per_proposal_cosine_5 = evaluate_per_proposal_cosine(similarity_cosine_df, true_label_df, top_n=5)
eval_per_proposal_cosine_7 = evaluate_per_proposal_cosine(similarity_cosine_df, true_label_df, top_n=7)
eval_per_proposal_cosine_10 = evaluate_per_proposal_cosine(similarity_cosine_df, true_label_df, top_n=10)

# eval_per_proposal_cosine_3.to_csv("/content/drive/MyDrive/Skripsi4/topik/cosine/30/hasil_eval_3_cosine_topik.csv", index=False)
# eval_per_proposal_cosine_5.to_csv("/content/drive/MyDrive/Skripsi4/topik/cosine/30/hasil_eval_5_cosine_topik.csv", index=False)
# eval_per_proposal_cosine_7.to_csv("/content/drive/MyDrive/Skripsi4/topik/cosine/30/hasil_eval_7_cosine_topik.csv", index=False)
# eval_per_proposal_cosine_10.to_csv("/content/drive/MyDrive/Skripsi4/topik/cosine/30/hasil_eval_10_cosine_topik.csv", index=False)